In [1]:
%%writefile mapper1.py
#!/usr/bin/env python3

import sys
import re

# print('reporter:counter:Personal Counters,Mapper started,' + str(1), file=sys.stderr)

for line in sys.stdin:
    # print('reporter:counter:Personal Counters,Lines in,' + str(1), file=sys.stderr)

    article_id, text = line.strip().split('\t', 1)
#     text = re.sub('[^\w\s]', ' ', text)

    try:
        words = re.split('\W*\s+\W*', text.strip())
    except:
        continue

    for word in words:
        # print('reporter:counter:Personal Counters,Words in,' + str(1), file=sys.stderr)
        capitalised = None
        try:
            if not word[0].islower() and word[1:].islower():
                capitalised = 1
            else:
                capitalised = 0
        except:
            continue

        if not word[0].isdigit() and capitalised is not None:
            # print('reporter:counter:Personal Counters,Total words,' + str(1), file=sys.stderr)
            # print('reporter:counter:Personal Counters,Capitalized words,' + str(capitalised), file=sys.stderr)
            print(word.lower() + '\t' + str(1) + '\t' + str(capitalised))


Overwriting mapper1.py


In [2]:
%%writefile combiner1.py
#!/usr/bin/env python3

import sys

current_key = None
word_sum = 0
capitalized_sum = 0

for line in sys.stdin:
    # print('reporter:counter:Personal Counters,Combiner in,' + str(1), file=sys.stderr)
    try:
        key, count, capitalized_count = line.strip().split('\t', 2)
        count = int(count)
        capitalized_count = int(capitalized_count)
    # print('capitalized count: ', str(capitalized_count))
    except ValueError as e:
        continue

    if current_key != key:
        if current_key:
            print(str(current_key) + '\t' + str(word_sum) + '\t' + str(capitalized_sum))
            # print('reporter:counter:Personal Counters,Combiner out,' + str(1), file=sys.stderr)

        word_sum = count
        capitalized_sum = capitalized_count
        current_key = key
    else:
        word_sum += count
        capitalized_sum += capitalized_count

if current_key:
    print(str(current_key) + '\t' + str(word_sum) + '\t' + str(capitalized_sum))
    # print('reporter:counter:Personal Counters,Combiner out,' + str(1), file=sys.stderr)


Overwriting combiner1.py


In [3]:
%%writefile reducer1.py
#!/usr/bin/env python3

import sys

current_key = None
word_sum = 0
capitalized_sum = 0

for line in sys.stdin:
    try:
        key, count, capitalized_count = line.strip().split('\t', 2)
        count = int(count)
        capitalized_count = int(capitalized_count)

    except ValueError as e:
        continue

    if current_key != key:
        if current_key:
            print(str(current_key) + '\t' + str(word_sum) + '\t' + str(capitalized_sum))
            # print('reporter:counter:Personal Counters,Reducer out,' + str(1), file=sys.stderr)
        
        word_sum = count
        capitalized_sum = capitalized_count
        current_key = key
    else:
        word_sum += count
        capitalized_sum += capitalized_count

if current_key:
    print(str(current_key) + '\t' + str(word_sum) + '\t' + str(capitalized_sum))
    # print('reporter:counter:Personal Counters,Reducer out,' + str(1), file=sys.stderr)



Overwriting reducer1.py


In [4]:
%%writefile mapper2.py
#!/usr/bin/env python3

import sys

current_key = None
word_sum = 0
capitalized_sum = 0

for line in sys.stdin:
    try:
        key, count, capitalized_count = line.strip().split('\t', 2)
        count = int(count)
        capitalized_count = int(capitalized_count)

    except ValueError as e:
        continue

    if current_key != key:
        if current_key:
            print(str(word_sum) + '\t' + str(capitalized_sum) + '\t' + str(current_key))
            # print('reporter:counter:Personal Counters,Mappertwo out,' + str(1), file=sys.stderr)
        
        word_sum = count
        capitalized_sum = capitalized_count
        current_key = key
    else:
        word_sum += count
        capitalized_sum += capitalized_count

if current_key:
    print(str(word_sum) + '\t' + str(capitalized_sum) + '\t' + str(current_key))
    # print('reporter:counter:Personal Counters,Mappertwo out,' + str(1), file=sys.stderr)


Overwriting mapper2.py


In [5]:
%%writefile reducer2.py
# #!/usr/bin/env python3

import sys

total_count = 0
total_caps = 0
current_word = None

for line in sys.stdin:

    count, caps_count, key = line.strip().split('\t', 2)
    count = float(count)
    caps_count = float(caps_count)

    if key == current_word:
        total_caps += caps_count
        total_count += count
    else:
        if current_word is None:
            current_word = key
            total_caps = caps_count
            total_count = count

        else:
            # print(f'{current_word} total caps: {total_caps}, total count: {total_count}')
            caps_ratio = float(total_caps) / float(total_count)
            # print(f'{current_word} caps ratio:{caps_ratio}')
            if caps_ratio > 0.995:
                print(str(current_word) + '\t' + str(int(total_caps)))

            total_count = count
            total_caps = caps_count
            current_word = key

caps_ratio = float(total_caps) / float(total_count)
if caps_ratio > 0.995:
    print(str(current_word) + '\t' + str(int(total_caps)))

Overwriting reducer2.py


In [6]:
%%bash

OUT_DIR="assignment3_"$(date +"%s%6N")
OUT_DIR2="assignment3_b_"$(date +"%s%6N")
NUM_REDUCERS=4
REDUCER_OUTPUT="reducer_output.txt"

# Code for your first job
yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -files mapper1.py,reducer1.py,combiner1.py \
    -mapper 'python3 mapper1.py' \
    -combiner 'python3 combiner1.py' \
    -reducer 'python3 reducer1.py' \
    -numReduceTasks ${NUM_REDUCERS} \
    -input /data/wiki/en_articles_part \
    -output ${OUT_DIR} > /dev/null

# Code for your second job
yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapreduce.job.output.key.comparator.class=org.apache.hadoop.mapreduce.lib.partition.KeyFieldBasedComparator \
    -D mapreduce.partition.keycomparator.options="-k1,3nr" \
    -files mapper2.py,reducer2.py \
    -mapper 'python3 mapper2.py' \
    -reducer 'python3 reducer2.py' \
    -numReduceTasks 1 \
    -input ${OUT_DIR} \
    -output ${OUT_DIR2} > /dev/null

# Code for obtaining the results
hdfs dfs -cat ${OUT_DIR2}/part-00000 | sed -n "5p;8q"

hdfs dfs -rm -r -skipTrash ${OUT_DIR}* > /dev/null
hdfs dfs -rm -r -skipTrash ${OUT_DIR2}* > /dev/null

french	5741


18/09/16 20:09:24 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
18/09/16 20:09:24 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
18/09/16 20:09:25 INFO mapred.FileInputFormat: Total input files to process : 1
18/09/16 20:09:25 INFO mapreduce.JobSubmitter: number of splits:2
18/09/16 20:09:25 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1537124312840_0011
18/09/16 20:09:25 INFO impl.YarnClientImpl: Submitted application application_1537124312840_0011
18/09/16 20:09:25 INFO mapreduce.Job: The url to track the job: http://72cab25abe44:8088/proxy/application_1537124312840_0011/
18/09/16 20:09:25 INFO mapreduce.Job: Running job: job_1537124312840_0011
18/09/16 20:09:31 INFO mapreduce.Job: Job job_1537124312840_0011 running in uber mode : false
18/09/16 20:09:31 INFO mapreduce.Job:  map 0% reduce 0%
18/09/16 20:09:47 INFO mapreduce.Job:  map 45% reduce 0%
18/09/16 20:09:59 INFO mapreduce.Job:  map 63% reduce 0%
18/09/16 20:10:05 INFO 